In [78]:
!pip install tensorflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [79]:
# Import Tensorflow & Pathlib librairies
import tensorflow as tf 
import pathlib 
import pandas as pd 
import string
import shutil
import re
import os
import io
import warnings
warnings.filterwarnings('ignore')

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization



In [ ]:
# Import dataset with Pandas 

In [80]:
from google.colab import drive
drive.mount('/content/drive')
path = 'drive/MyDrive/bloc4/' 

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [81]:
dataset = pd.read_csv(path+"spam (2).csv",encoding="ISO-8859-1", usecols=[0,1])
dataset.columns = ['type', 'content']

In [82]:
dataset.head()

,type,content
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


structure du dataset que nous allons utilisé

In [ ]:
dataset.shape

(5572, 2)

Le but de notre modèle est de construire un détecteur de spam, qui peut automatiquement signaler les spams au fur et à mesure qu'ils viennent en fonction du contenu des SMS.
Approche Supervisé pour entraîner Le EMBEDDING 

Preprocessing

Nous allons maintenant passer par une phase de prétraitement. Le but est de nettoyer les chaînes de caractères et d'encoder les mots afin qu'ils soient représentés sous forme d'entiers.


1. Utilisez la commande : !python -m spacy download fr_core_news_sm pour télécharger tous les éléments de langue liés à la langue anglaise

In [83]:
!pip install -U pip setuptools wheel 
!pip install -U spacy
!python -m spacy download en_core_web_sm -q


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
2023-01-15 13:15:12.653190: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 42.9 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


2. Load now fr_core_news_sm

In [84]:
# Import Spacy and english initialisation
import en_core_web_sm
nlp = en_core_web_sm.load()


 3 Import the english STOP_WORDS

In [85]:
# Import Stop words 
from spacy.lang.en.stop_words import STOP_WORDS


4 Nous allons maintenant devoir nettoyer nos textes afin de les préparer pour la formation
. Faisons cela en trois étapes différentes :

en utilisant la commande str.isalnum, je vais supprimer tous les caractères des chaînes qui ne sont pas alphanumériques, à l'exception des espaces blancs.
en utilisant str.replace, str.lower et str.strip, j'ai remplacé les espaces blancs doubles par des espaces blancs simples, converti tous les caractères en minuscules et coupé les espaces blancs de début et de fin.
en utilisant spacy, et  par lemma_ et supprimez tous les mots vides.

In [86]:
### 

# DO NOT RUN THIS COMMAND (TAKES TIME)
# rather explain it and import the cleaned dataset in the next cell

###
dataset["content_clean"] = dataset["content"].apply(lambda x:''.join(ch for ch in x if ch.isalnum() or ch==" "))
dataset["content_clean"] = dataset["content_clean"].apply(lambda x: x.replace(" +"," ").lower().strip())
dataset["content_clean"] = dataset["content_clean"].apply(lambda x: " ".join([token.lemma_ for token in nlp(x) if (token.lemma_ not in STOP_WORDS) & (token.text not in STOP_WORDS)]))

dataset

,type,content,content_clean
0,ham,"Go until jurong point, crazy.. Available only ...",jurong point crazy available bugis n great wor...
1,ham,Ok lar... Joking wif u oni...,ok lar joking wif u oni
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,free entry 2 wkly comp win fa cup final tkts 2...
3,ham,U dun say so early hor... U c already then say...,u dun early hor u c
4,ham,"Nah I don't think he goes to usf, he lives aro...",nah think usf live
...,...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...,2nd time try 2 contact u u win å750 pound priz...
5568,ham,Will Ì_ b going to esplanade fr home?,ì b esplanade fr home
5569,ham,"Pity, * was in mood for that. So...any other s...",pity mood soany suggestion
5570,ham,The guy did some bitching but I acted like i'd...,guy bitching act like d interested buy week free


In [87]:
dataset['type'].value_counts()

ham     4825
spam     747
Name: type, dtype: int64

In [88]:
dataset['target'] = dataset['type'].apply(lambda x : 1 if x=='spam' else 0)
dataset.head()

,type,content,content_clean,target
0,ham,"Go until jurong point, crazy.. Available only ...",jurong point crazy available bugis n great wor...,0
1,ham,Ok lar... Joking wif u oni...,ok lar joking wif u oni,0
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,free entry 2 wkly comp win fa cup final tkts 2...,1
3,ham,U dun say so early hor... U c already then say...,u dun early hor u c,0
4,ham,"Nah I don't think he goes to usf, he lives aro...",nah think usf live,0


5.  À l'aide de tf.keras.preprocessing.text.Tokenizer Tokenizer, j'ai procèdé à l'encodage de tous les avis (attention, certains avis peuvent avoir été entièrement effacés, essayez de comprendre pourquoi, supprimez ces avis)

Lors de l'instanciation du tokenizer, je me suis assuré de le configurer pour ne conserver que les 1000 mots les plus courants.

In [89]:
import numpy as np
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=1000) # instanciate the tokenizer
tokenizer.fit_on_texts(dataset["content_clean"])
dataset["content_encoded"] = tokenizer.texts_to_sequences(dataset.content_clean)
dataset["len_content"] = dataset["content_encoded"].apply(lambda x: len(x))
dataset = dataset[dataset["len_content"]!=0]
dataset.head()

,type,content,content_clean,target,content_encoded,len_content
0,ham,"Go until jurong point, crazy.. Available only ...",jurong point crazy available bugis n great wor...,0,"[230, 444, 460, 943, 35, 51, 204, 944, 79, 945...",11
1,ham,Ok lar... Joking wif u oni...,ok lar joking wif u oni,0,"[9, 193, 289, 1]",4
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,free entry 2 wkly comp win fa cup final tkts 2...,1,"[11, 300, 3, 532, 655, 33, 849, 420, 20, 157, ...",13
3,ham,U dun say so early hor... U c already then say...,u dun early hor u c,0,"[1, 124, 149, 1, 84]",5
4,ham,"Nah I don't think he goes to usf, he lives aro...",nah think usf live,0,"[705, 22, 656, 127]",4


6 .Tensorflow est incapable pour l'instant de créer un jeu de données tenseur basé sur des listes de longueurs différentes, j'ai stocké tous les textes encodés dans un seul tableau numpy avant de créer le jeu de données tensorflow. Toutes nos séquences n'ont pas la même longueur, c'est là que le tf.keras.preprocessing.sequence.pad_sequences est utile, il ajoutera zéro remplissage au début (padding="pre") ou à la fin (padding="post ") de nos séquences afin qu'elles aient toutes la même longueur. Complétons les séquences.

In [90]:
content_pad = tf.keras.preprocessing.sequence.pad_sequences(dataset.content_encoded, padding="post")

7 Maintenant que mes séquences sont remplies, je vais crée le jeu de données tenseur, et je vais  commencer par traiter la variable target comme catégorielle, 

In [91]:
full_ds = tf.data.Dataset.from_tensor_slices((content_pad, dataset.target.values))

8 J'ai effectué un train_test_split de vos données (gardez environ 70 % dans le train). Pour cela, j'ai  utilisé les méthodes .take et .skip sur l'ensemble de données tensorflow. Une fois que vous j'ai fais cela, j'ai  utilisé .shuffle sur le train et .batch sur les deux ensembles pour les organiser par lots de 64 observations.

In [92]:
# Train Test Split
TAKE_SIZE = int(0.7*dataset.shape[0])

train_data = full_ds.take(TAKE_SIZE).shuffle(TAKE_SIZE)
train_data = train_data.batch(64)

test_data = full_ds.skip(TAKE_SIZE)
test_data = test_data.batch(64)

9 Look at a batch of data

In [93]:
 # Regardons un batch 
for content, target in train_data.take(1):
  print(content, target)

tf.Tensor(
[[235 626 164 ...   0   0   0]
 [  6 376 393 ...   0   0   0]
 [ 85 237  51 ...   0   0   0]
 ...
 [  8   0   0 ...   0   0   0]
 [132  53   7 ...   0   0   0]
 [667 416 489 ...   0   0   0]], shape=(64, 47), dtype=int32) tf.Tensor(
[0 0 1 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0
 1 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 1 0 0 0 0 0 1], shape=(64,), dtype=int64)


# Classification Modeling


Nous commencerons par traiter l'analyse des spams comme un problème de classification (cela affectera la dernière couche et le choix de la fonction de perte et de la métrique).

##SIMPLE RNN

In [94]:
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense, GRU, LSTM, Dropout

vocab_size = len(tokenizer.word_index)
model = tf.keras.Sequential([
                  # Word Embedding layer           
                  Embedding(vocab_size+1, 32, input_shape=[content.shape[1],],name="embedding"),
                  # Gobal average pooling
                  ##SimpleRNN(units=32, return_sequences=True), # maintains the sequential nature
                  SimpleRNN(units=16, return_sequences=False), # returns the last output
                  # Dense layers once the data is flat
                  Dropout(0.2),              
                  Dense (8, activation='relu'),

                  # output layer with as many neurons as the number of classes
                  # for the target variable and softmax activation
                  Dense(1, activation="sigmoid")
])

In [95]:
model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 47, 32)            262048    
                                                                 
 simple_rnn_3 (SimpleRNN)    (None, 16)                784       
                                                                 
 dropout_3 (Dropout)         (None, 16)                0         
                                                                 
 dense_14 (Dense)            (None, 8)                 136       
                                                                 
 dense_15 (Dense)            (None, 1)                 9         
                                                                 
Total params: 262,977
Trainable params: 262,977
Non-trainable params: 0
_________________________________________________________________


In [96]:
optimizer= tf.keras.optimizers.Adam(0.001)

model.compile(optimizer=optimizer,
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=['accuracy'])

A l'aide de la fonction pandas .value_counts créez un dictionnaire qui attribue à chaque valeur de la variable cible un poids inversement proportionnel à leur fréquence dans le jeu de données. Plus précisément, calculez l'inverse de la fréquence pour chaque classe puis divisez par 2 et multipliez par le nombre d'observations.

In [97]:
weights = 1/(dataset["target"]).value_counts()
weights = weights * len(dataset)/2
weights = {index : values for index , values in zip(weights.index,weights.values)}
weights

{0: 0.5798157274480394, 1: 3.632214765100671}


Ajustez votre modèle sur 30 époques avec des poids pour pénaliser les notes trop fréquentes.

In [98]:
# Model training 
es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
model.fit(train_data,
          epochs=30, 
          validation_data=test_data,
          class_weight=weights,
         callbacks=[es_callback])

Epoch 1/30
60/60 [==============================] - 3s 23ms/step - loss: 0.6048 - accuracy: 0.7944 - val_loss: 0.3080 - val_accuracy: 0.9107
Epoch 2/30
60/60 [==============================] - 1s 18ms/step - loss: 0.3758 - accuracy: 0.9409 - val_loss: 0.3590 - val_accuracy: 0.8541
Epoch 3/30
60/60 [==============================] - 2s 29ms/step - loss: 0.2323 - accuracy: 0.9615 - val_loss: 0.1341 - val_accuracy: 0.9563
Epoch 4/30
60/60 [==============================] - 2s 27ms/step - loss: 0.1385 - accuracy: 0.9723 - val_loss: 0.1678 - val_accuracy: 0.9477
Epoch 5/30
60/60 [==============================] - 1s 18ms/step - loss: 0.1037 - accuracy: 0.9768 - val_loss: 0.1351 - val_accuracy: 0.9563
Epoch 6/30
60/60 [==============================] - 1s 18ms/step - loss: 0.0661 - accuracy: 0.9879 - val_loss: 0.1630 - val_accuracy: 0.9427


### GRU

1. Créez un objet nommé model_gru en remplaçant les couches SimpleRNN par des couches GRU et reproduisez les mêmes étapes.

In [104]:


vocab_size = len(tokenizer.word_index)
model_gru = tf.keras.Sequential([
                  Embedding(vocab_size+1, 32, input_shape=[content.shape[1],],name="embedding"),
                  GRU(units=32, return_sequences=True), # maintains the sequential nature
                  GRU(units=16, return_sequences=False), # returns the last output
                  
                  Dense(8, activation='relu'),

                  Dense(1, activation="sigmoid")])

In [105]:
model_gru.summary()

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 47, 32)            262048    
                                                                 
 gru_8 (GRU)                 (None, 47, 32)            6336      
                                                                 
 gru_9 (GRU)                 (None, 16)                2400      
                                                                 
 dense_18 (Dense)            (None, 8)                 136       
                                                                 
 dense_19 (Dense)            (None, 1)                 9         
                                                                 
Total params: 270,929
Trainable params: 270,929
Non-trainable params: 0
_________________________________________________________________


In [106]:
optimizer= tf.keras.optimizers.Adam(0.00001)

model_gru.compile(optimizer=optimizer,
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=['accuracy'])

In [107]:
# Model training 
es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
model_gru.fit(train_data,
          epochs=30, 
          validation_data=test_data,
          class_weight=weights,
         callbacks=[es_callback])

Epoch 1/30
60/60 [==============================] - 10s 74ms/step - loss: 0.6923 - accuracy: 0.8627 - val_loss: 0.6919 - val_accuracy: 0.8615
Epoch 2/30
60/60 [==============================] - 4s 60ms/step - loss: 0.6923 - accuracy: 0.8627 - val_loss: 0.6918 - val_accuracy: 0.8615
Epoch 3/30
60/60 [==============================] - 4s 60ms/step - loss: 0.6923 - accuracy: 0.8627 - val_loss: 0.6918 - val_accuracy: 0.8615
Epoch 4/30
60/60 [==============================] - 4s 60ms/step - loss: 0.6923 - accuracy: 0.8627 - val_loss: 0.6920 - val_accuracy: 0.8615
Epoch 5/30
60/60 [==============================] - 4s 59ms/step - loss: 0.6923 - accuracy: 0.8627 - val_loss: 0.6918 - val_accuracy: 0.8615
Epoch 6/30
60/60 [==============================] - 4s 59ms/step - loss: 0.6923 - accuracy: 0.8627 - val_loss: 0.6919 - val_accuracy: 0.8615
Epoch 7/30
60/60 [==============================] - 4s 60ms/step - loss: 0.6923 - accuracy: 0.8627 - val_loss: 0.6919 - val_accuracy: 0.8615


## LSTM
Créez un objet nommé model_lstm en remplaçant les couches SimpleRNN par des couches LSTM et reproduisez les mêmes étapes.



In [108]:

vocab_size = len(tokenizer.word_index)
model_lstm  = tf.keras.Sequential([
                  Embedding(vocab_size+1, 32, input_shape=[content.shape[1],],name="embedding"),
                  GRU(units=32, return_sequences=True), # maintains the sequential nature
                  GRU(units=16, return_sequences=False), # returns the last output
                  
                  Dense(8, activation='relu'),

                  Dense(1, activation="sigmoid")])

In [109]:
model_lstm.summary()

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 47, 32)            262048    
                                                                 
 gru_10 (GRU)                (None, 47, 32)            6336      
                                                                 
 gru_11 (GRU)                (None, 16)                2400      
                                                                 
 dense_20 (Dense)            (None, 8)                 136       
                                                                 
 dense_21 (Dense)            (None, 1)                 9         
                                                                 
Total params: 270,929
Trainable params: 270,929
Non-trainable params: 0
_________________________________________________________________


In [110]:
optimizer= tf.keras.optimizers.Adam(0.0001)

model_lstm.compile(optimizer=optimizer,
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=['accuracy'])

In [111]:
# Model training 
es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
model_lstm.fit(train_data,
          epochs=30, 
          validation_data=test_data,
          class_weight=weights,
         callbacks=[es_callback])

Epoch 1/30
60/60 [==============================] - 9s 73ms/step - loss: 0.6924 - accuracy: 0.6357 - val_loss: 0.6924 - val_accuracy: 0.8608
Epoch 2/30
60/60 [==============================] - 4s 58ms/step - loss: 0.6924 - accuracy: 0.8619 - val_loss: 0.6907 - val_accuracy: 0.8608
Epoch 3/30
60/60 [==============================] - 4s 61ms/step - loss: 0.6924 - accuracy: 0.8619 - val_loss: 0.6904 - val_accuracy: 0.8615
Epoch 4/30
60/60 [==============================] - 3s 58ms/step - loss: 0.6924 - accuracy: 0.8627 - val_loss: 0.6920 - val_accuracy: 0.8615
Epoch 5/30
60/60 [==============================] - 3s 58ms/step - loss: 0.6924 - accuracy: 0.8627 - val_loss: 0.6913 - val_accuracy: 0.8615
Epoch 6/30
60/60 [==============================] - 4s 59ms/step - loss: 0.6924 - accuracy: 0.8627 - val_loss: 0.6916 - val_accuracy: 0.8615
Epoch 7/30
60/60 [==============================] - 4s 60ms/step - loss: 0.6924 - accuracy: 0.8627 - val_loss: 0.6894 - val_accuracy: 0.8615
Epoch 8/30
60

# Classification Evaluation

Classification Evaluation

This part will focus on visualizing the training process and interpreting the results for our predictive models.
SimpleRNN

    Create a graph showing your loss and validation loss in relation to the number of epochs for the simpleRNN model.



In [112]:
import plotly.graph_objects as go
fig = go.Figure()
fig.add_trace(go.Scatter(y=model.history.history["loss"],
                    mode='lines',
                    name='loss'))
fig.add_trace(go.Scatter(y=model.history.history["val_loss"],
                    mode='lines',
                    name='val_loss'))
fig.show()

In [113]:

fig = go.Figure()
fig.add_trace(go.Scatter(y=model.history.history["accuracy"],
                    mode='lines',
                    name='accuracy'))
fig.add_trace(go.Scatter(y=model.history.history["val_accuracy"],
                    mode='lines',
                    name='val_accuracy'))
fig.show()

In [120]:
GRU

keras.layers.rnn.gru.GRU

In [121]:
import plotly.graph_objects as go
fig = go.Figure()
fig.add_trace(go.Scatter(y=model_gru.history.history["loss"],
                    mode='lines',
                    name='loss'))
fig.add_trace(go.Scatter(y=model_gru.history.history["val_loss"],
                    mode='lines',
                    name='val_loss'))
fig.show()

In [122]:
fig = go.Figure()
fig.add_trace(go.Scatter(y=model_gru.history.history["accuracy"],
                    mode='lines',
                    name='accuracy'))
fig.add_trace(go.Scatter(y=model_gru.history.history["val_accuracy"],
                    mode='lines',
                    name='val_accuracy'))
fig.show()

In [126]:
LSTM

keras.layers.rnn.lstm.LSTM

In [127]:
import plotly.graph_objects as go
fig = go.Figure()
fig.add_trace(go.Scatter(y=model_gru.history.history["loss"],
                    mode='lines',
                    name='loss'))
fig.add_trace(go.Scatter(y=model_gru.history.history["val_loss"],
                    mode='lines',
                    name='val_loss'))
fig.show()

In [128]:
fig = go.Figure()
fig.add_trace(go.Scatter(y=model_lstm.history.history["accuracy"],
                    mode='lines',
                    name='accuracy'))
fig.add_trace(go.Scatter(y=model_lstm.history.history["val_accuracy"],
                    mode='lines',
                    name='val_accuracy'))
fig.show()

# transferlearning

In [ ]:
from sklearn.model_selection import train_test_split
 X_train, X_val, y_train, y_val = train_test_split(dataset['content_clean'], dataset['target'], test_size=0.33, random_state=42, stratify=dataset.target.values)

In [ ]:
import tensorflow_hub as hub
module_url = "https://tfhub.dev/google/universal-sentence-encoder-large/5"
hub_layer=hub.KerasLayer(module_url,input_shape=[],output_shape=[47],dtype=tf.string,trainable=False)

In [ ]:
model=tf.keras.models.Sequential([
    hub_layer, 
    Dense(8,activation='relu'),
    Dense(1,activation='sigmoid')
])

model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 512)               147354880 
                                                                 
 dense_6 (Dense)             (None, 8)                 4104      
                                                                 
 dense_7 (Dense)             (None, 1)                 9         
                                                                 
Total params: 147,358,993
Trainable params: 4,113
Non-trainable params: 147,354,880
_________________________________________________________________


In [ ]:
optimizer = tf.keras.optimizers.Adam(0.0001)
model.compile(optimizer=optimizer,
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=['accuracy'])

es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

model.fit(
    X_train,
    y_train,
    epochs=50,
    batch_size=32,
    validation_data=(X_val, y_val),
    callbacks=es_callback)